This notebook shows a simple example of inverse kinematics (as seen in class) for the Kuka iiwa robot using a QP instead of the pseudo-inverse.

The notebook contains:
* an example for end-effector position goals only
* an example for end-effector position goals with optimization of a posture in the nullspace
* an example for end-effector pose (positions + orientation) with nullspace posture optimization

## Requirement
To run these examples you will need to install these two packages:
* [Pinocchio](https://github.com/stack-of-tasks/pinocchio), a rigid body dynamics library (which does all the kinematics computation for us). You can install it directly through conda `conda install pinocchio -c conda-forge` or follow the [installation instructions](https://stack-of-tasks.github.io/pinocchio/download.html#Install_3)
* [Meshcat](https://github.com/rdeits/meshcat-python) which is used for the visualization of the robot. You can install it directly through pip `pip install meshcat`
* [ProxSuite](https://simple-robotics.github.io/proxsuite/) which is used for the QP solver. You can install it directly through conda `conda install proxsuite -c conda-forge`

In [5]:
import time
import numpy as np
import pinocchio as pin
import meshcat
import proxsuite

In [2]:
# the directory where the robot models are located
package_dirs = './urdf/'

# the name of the URDF model (the robot model)
urdf_file = 'iiwa.urdf'
END_EFF_FRAME_ID = 17 # number of the frame corresponding to the end-effector

# we load the urdf models with Pinocchio
urdf = package_dirs + urdf_file
robot = pin.RobotWrapper.BuildFromURDF(urdf, package_dirs)

# we create the visualization
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)

try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)
    
viz.loadViewerModel()

# place the robot in its default position and display it
q0 = pin.neutral(robot.model)
viz.display(q0)

# create a ball to visualize the goal
viz.viewer['ball'].set_object(meshcat.geometry.Sphere(0.05), 
                              meshcat.geometry.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


## inverse kinematics with end-effector goal position

Here we want to find the inverse kinematics of the robot to reach a desired end-effector position $p_{des}$ by using a QP solver. 

We setup 3 different possible QP (you can set the variable ``QP_type`` to 1, 2 or 3 to select which one to run).

### First QP option: simple task optimization
The first QP is simple, we simply optimize the following
$$\min_{\Delta\theta} \frac{1}{2}\|J \Delta\theta + p(\theta)_{current} - p_{des}\|^2$$
which is equivalent (dropping the constant terms) to optimizing
$$\min_{\Delta\theta} \frac{1}{2}\Delta\theta^T J^T J \Delta\theta + (p(\theta)_{current} - p_{des})^TJ\Delta\theta$$

we further add constraints to ensure we take a small step at each iteration $|\Delta \theta_i| < 0.1$

### Second QP option: adding a linear constraint on the second joint
The second QP adds a constraint to prevent the first joint from moving
   $$\min_{\Delta\theta} \frac{1}{2}\|J \Delta\theta + p(\theta)_{current} - p_{des}\|^2$$
   $$\textrm{subject to} \ \ \begin{bmatrix}1 & 0 & 0 & 0 & 0 & 0 & 0 \end{bmatrix} \Delta\theta = 0 \ \ \textrm{and} \ \ |\Delta \theta_i| < 0.1$$

We will see that in this case, the robot cannot reach its goal but will find the closest solution it can.

### Third QP option: adding a cost to optimize joint configurations as well
The third QP tries to both reach the desired end-effector position (high weight) and optimize the joint positions to reach a desired position
   $$\min_{\Delta\theta} \frac{1}{2}\left(1000*\|J \Delta\theta + p(\theta)_{current} - p_{des}\|^2 + \|\theta + \Delta\theta - \theta_{desired}\|^2 \right)$$

In [59]:
QP_type = 1

# we set the desired end-effector position (feel free to change this)
p_des = np.array([0.5,.6, 0.2])
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(p_des))

# we start with a random guess for the joint positions (q) 
# here we use the default robot position (for simplicity)
q = pin.neutral(robot.model)
viz.display(q)

## we iterate up to 500 times
max_iterations = 500
for i in range(max_iterations):
    # compute the forward kinematics
    robot.forwardKinematics(q)
    
    # get the position of the end-effector
    T_S_F = robot.framePlacement(q, END_EFF_FRAME_ID)
    p_current = T_S_F.translation
    
    # get the Jacobian with the right frame 
    # (we use a frame oriented like the spatial frame but located at the end-effector)
    J = robot.computeFrameJacobian(q, END_EFF_FRAME_ID)
    J = T_S_F.rotation @ J[:3,:]
    
    # compute the endeffector error
    err =  p_current - p_des
    
    # here we check if we are done or if we need to continue
    err_norm = np.linalg.norm(err)
    if err_norm < 0.005: # we stop if we have an error < 5mm
        print(f'done in {i} steps')
        print(f'we found joint positions:\n {q}')
        print(f'this gives an end-effector position of {T_S_F.translation}')
        print(f'the desired end-effector position was {p_des}')
        break
    if i==max_iterations-1:
        print('no good solution found')
        print(f'current guess is:\n {q}')
        print(f'this gives an end-effector position of {T_S_F.translation}')
        print(f'the desired end-effector position was {p_des}')

    ## we setup the QP
    if QP_type == 1:
        H = J.T @ J
        h = J.T @ err
        qp = proxsuite.proxqp.dense.QP(robot.nv, 0, 0, True)
        l_box = -np.ones(robot.nv) * 0.1 #lower limit
        u_box = np.ones(robot.nv) * 0.1 #upper limit
        qp.init(H, h, None, None, None, None, None, l_box, u_box)
    elif QP_type == 2:
        H = J.T @ J
        h = J.T @ err
        qp = proxsuite.proxqp.dense.QP(robot.nv, 1, 0, True)
        A = np.array([[1,0,0,0,0,0,0]])
        b = np.array([0])
        l_box = -np.ones(robot.nv) * 0.1 #lower limit
        u_box = np.ones(robot.nv) * 0.1 #upper limit
        qp.init(H, h, A, b, None, None, None, l_box, u_box)
    elif QP_type == 3:
        theta_desired = np.array([3., 3., -3, 0., -2., 1., 1.])
        # theta_desired = np.zeros([robot.nv])
        H = 1000 * J.T @ J + np.identity(robot.nv)
        h = 1000 * J.T @ err + (q - theta_desired)
        qp = proxsuite.proxqp.dense.QP(robot.nv, 0, 0)
        qp.init(H, h)
    else:
        print('ERROR this mode does not exist')
        raise
        
    qp.solve()
    dq = qp.results.x
    
    # compute do one step of diffential inverse kinematics
    epsilon = 10e-2
    # we update our guess for the joint positions
    q = q + epsilon * dq
    
    # we display the new guess (and sleep to make it nicely visible with rendering)
    # turn this off to see how fast this really is
    viz.display(q)
    time.sleep(0.001)


no good solution found
current guess is:
 [-5.58670200e-06  1.88917053e+00  1.57748291e+00 -1.39693957e+00
 -1.57903939e+00 -6.51670225e-03  0.00000000e+00]
this gives an end-effector position of [0.48580499 0.51312502 0.20432934]
the desired end-effector position was [0.5 0.6 0.2]


## inverse kinematics with full pose goal (position + orientation)
We can also optimize the full pose of the end-effector using the log by optimizing

$$\min_{\Delta\theta} \frac{1}{2}\|J \Delta\theta - \log(T_{current}^{-1} T_{des})\|^2$$

with constraint at each iteration $|\Delta \theta_i| < 0.1$

In [73]:
# inverse kinematics with nullspace

# we set the desired end-effector position and orientation (feel free to change this)
rot_des = np.array([[1,0,0],[0,1,0],[0,0,1]]) # desired orientation matrix
pos_des = np.array([0.3,.5, 0.5]) # desired position
end_eff_desired = pin.SE3(rot_des,pos_des) # we create a homogeneous transform

# we display the goal
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(end_eff_desired.translation)) 

# we start with a random guess for the joint positions (q) 
# here we use the default robot position
q = pin.neutral(robot.model)
viz.display(q)

## we iterate up to 500 times
max_iterations = 500
for i in range(max_iterations):
    # compute the forward kinematics
    robot.forwardKinematics(q)
    
    # get the position of the end-effector
    T_S_F = robot.framePlacement(q, END_EFF_FRAME_ID)
    
    # get the Jacobian with the right frame 
    # CAREFUL we use a body Jacobian for this task!
    J = robot.computeFrameJacobian(q, END_EFF_FRAME_ID)
    
    # compute the endeffector error
    # to get the error we use the log (implemented in Pinocchio)
    err = -pin.log(T_S_F.actInv(end_eff_desired)).vector
    
    # here we check if we are done or if we need to continue
    err_norm = np.linalg.norm(err)
    if err_norm < 0.01:
        print(f'done in {i} steps')
        print(f'we found joint positions:\n {q}')
        print(f'this gives an end-effector position of \n{T_S_F}')
        print(f'the desired end-effector position was \n{end_eff_desired}')
        break
    if i==max_iterations-1:
        print('no good solution found')
        print(f'current guess is:\n {q}')
        print(f'this gives an end-effector position of \n{T_S_F}')
        print(f'the desired end-effector position was \n{end_eff_desired}')

    ## solve the QP
    H = J.T @ J
    h = J.T @ err
    qp = proxsuite.proxqp.dense.QP(robot.nv, 0, 0, True)
    l_box = -np.ones(robot.nv) * 0.1 #lower limit
    u_box = np.ones(robot.nv) * 0.1 #upper limit
    qp.init(H, h, None, None, None, None, None, l_box, u_box)
    qp.solve()
    dq = qp.results.x
    
    # compute do one step of diffentiual inverse kinematics
    epsilon = 10e-2
    # we update our guess for the joint positions
    q = q + epsilon * dq
    
    # we display the new guess (and sleep to make it nicely visible with rendering)
    # turn this off to see how fast this really is
    viz.display(q)
    time.sleep(0.001)


done in 315 steps
we found joint positions:
 [ 1.52943595  2.10437844  0.78963727  1.52460117 -0.85008383 -0.95053851
 -0.46981443]
this gives an end-effector position of 
  R =
           1 -9.97785e-05 -0.000613217
 9.91087e-05     0.999999  -0.00109231
 0.000613325   0.00109225     0.999999
  p = 0.304946 0.508306 0.500216

the desired end-effector position was 
  R =
1 0 0
0 1 0
0 0 1
  p = 0.3 0.5 0.5

